In [1]:
import torch
torch.cuda.is_available()
print(torch.zeros(1).cuda())

import torch_geometric
import torch_geometric.nn as pyg_nn
from torch_sparse import SparseTensor, matmul
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
print(torch_geometric.__version__)

import torch_scatter
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.utils as pyg_utils

from torch import Tensor
from typing import Union, Tuple, Optional
from torch_geometric.typing import (OptPairTensor, Adj, Size, NoneType, OptTensor)

from torch.nn import Parameter, Linear
from torch_sparse import SparseTensor, set_diag
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax

import networkx as nx
from deepsnap.hetero_graph import HeteroGraph
import copy
from copy import deepcopy
import pickle

import deepsnap
from deepsnap.graph import Graph
from deepsnap.batch import Batch
from deepsnap.dataset import GraphDataset
from deepsnap.hetero_gnn import forward_op
from deepsnap.hetero_graph import HeteroGraph
from sklearn.metrics import f1_score, roc_auc_score

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import json
from pathlib import Path as Data_Path
from os import listdir
from os.path import isfile, join
from itertools import combinations
from tqdm.notebook import tqdm

import graph_tool.all as gt
import json
print("graph-tool version: {}".format(gt.__version__.split(' ')[0]))

tensor([0.], device='cuda:0')
2.3.0
graph-tool version: 2.44


In [2]:
import pickle

with open("/home/asa489/Downloads/updated_fixed_graph.pickle", "rb") as f:
    g_nx = pickle.load(f)

### N-hop neighbourhood

In [ ]:
pass

### Deepsnap

In [3]:
# Create a DeepSNAP graph from NetworkX graph
ds_graph = Graph(g_nx)

/home/asa489/.conda/envs/ayush_project/lib/python3.10/site-packages/deepsnap/graph.py:522: UserWarning: Node related key is required.
  warnings.warn("Node related key is required.")


In [4]:
print("Number of vertices:", g_nx.number_of_nodes())
print("Number of edges:", g_nx.number_of_edges())

Number of vertices: 690613
Number of edges: 6655333


In [5]:
task = 'link_pred'
dataset = GraphDataset([ds_graph], task=task, edge_train_mode='disjoint')

dataset_train, dataset_val, dataset_test = dataset.split(transductive=True, split_ratio=[0.8, 0.1, 0.1])

# dataset_train[0].to('cuda')
# dataset_val[0].to('cuda')
# dataset_test[0].to('cuda')

# dataset_train.to('cuda:0')
# dataset_val.to('cuda:0')
# dataset_test.to('cuda:0')

print(type(dataset_train))
print(dataset_train[0])
print(type(dataset_train[0]))

num_train_edges = dataset_train[0].edge_label_index.shape[1]
num_val_edges = dataset_val[0].edge_label_index.shape[1]
num_test_edges = dataset_test[0].edge_label_index.shape[1]

print("Train set has {} supervision (positive) edges".format(num_train_edges // 2))
print("Validation set has {} supervision (positive) edges".format(num_val_edges // 2))
print("Test set has {} supervision (positive) edges".format(num_test_edges // 2))

print("Train set has {} message passing edges".format(dataset_train[0].edge_index.shape[1]))
print("Validation set has {} message passing edges".format(dataset_val[0].edge_index.shape[1]))
print("Test set has {} message passing edges".format(dataset_test[0].edge_index.shape[1]))

print(dataset)

<class 'deepsnap.dataset.GraphDataset'>
Graph(G=[], edge_index=[2, 8518824], edge_label=[4259416], edge_label_index=[2, 4259416], name=[690613], negative_label_val=[1], node_label_index=[690613], type=[690613], uri=[690613])
<class 'deepsnap.graph.Graph'>
Train set has 2129708 supervision (positive) edges
Validation set has 1331066 supervision (positive) edges
Test set has 1331068 supervision (positive) edges
Train set has 8518824 message passing edges
Validation set has 10648532 message passing edges
Test set has 11979598 message passing edges
GraphDataset(1)


In [7]:
pickle.dump(dataset_train, open('./fixed_graphs/train.graph', 'wb'))

In [8]:
pickle.dump(dataset_val, open('./fixed_graphs/val.graph', 'wb'))

In [9]:
pickle.dump(dataset_test, open('./fixed_graphs/test.graph', 'wb'))

In [2]:
dataset_train = pickle.load(open('/home/asa489/fixed_graphs/train.graph', 'rb'))

In [3]:
dataset_val = pickle.load(open('/home/asa489/fixed_graphs/val.graph', 'rb'))

In [12]:
#dataset_test = pickle.load(open('/home/asd27/scratch/fixed_graphs/test.graph', 'rb'))

In [4]:
class LightGCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels, normalize = True,
                 bias = False, **kwargs):  
        super(LightGCNConv, self).__init__(**kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.normalize = normalize

    def forward(self, x, edge_index, size = None):
        out = self.propagate(edge_index, x=(x, x))
        return out

    def message(self, x_j):
        out = x_j
        return out

    def aggregate(self, inputs, index, dim_size = None):
        node_dim = self.node_dim
        out = torch_scatter.scatter(inputs, index, dim=node_dim, reduce='mean')
        return out

In [5]:
class LightGCN(torch.nn.Module):
    def __init__(self, train_data, num_layers, emb_size=16, initialize_with_words=False):
        super(LightGCN, self).__init__()
        self.convs = nn.ModuleList()
        assert (num_layers >= 1), 'Number of layers is not >=1'
        for l in range(num_layers):
            self.convs.append(LightGCNConv(input_dim, input_dim))

        # Initialize using custom embeddings if provided
        num_nodes = train_data.node_label_index.size()[0]
        self.embeddings = nn.Embedding(num_nodes, emb_size)
        if initialize_with_words:
            self.embeddings.weight.data.copy_(train_datanode_features)
        
        self.loss_fn = nn.BCELoss()
        self.num_layers = num_layers
        self.emb_size = emb_size
        self.num_modes = num_nodes

    def forward(self, data):
        edge_index, edge_label_index, node_label_index = data.edge_index, data.edge_label_index, data.node_label_index
        layer_embeddings = []
        
        x = self.embeddings(node_label_index)
        mean_layer = x

        # We take an average of ever layer's node embeddings
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)
            # print("x shape",x.shape)
            # print("mean_layer shape",mean_layer.shape)
            mean_layer += x

        mean_layer /= 4

        # Prediction head is simply dot product
        nodes_first = torch.index_select(x, 0, edge_label_index[0,:].long())
        nodes_second = torch.index_select(x, 0, edge_label_index[1,:].long())

        # Since we don't want a rank output, we create a sigmoid of the dot product
        out = torch.sum(nodes_first * nodes_second, dim=-1) # FOR RANKING
        pred = torch.sigmoid(out)

        return torch.flatten(pred)

    def loss(self, pred, label):
        return self.loss_fn(pred, label)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
args = {
    'device' : 'cuda' if torch.cuda.is_available() else 'cpu',
    'num_layers' : 3,
    'emb_size' : 32,
    'weight_decay': 1e-6,
    'lr': 0.4,
    'epochs': 600
}

    

In [7]:
   
datasets = {
    'train': dataset_train[0],
    'val': dataset_val[0],
#     'test': dataset_test[0]
}     
input_dim = datasets['train'].num_node_features
print(input_dim, args)

0 {'device': 'cuda', 'num_layers': 3, 'emb_size': 32, 'weight_decay': 1e-06, 'lr': 0.4, 'epochs': 600}


In [80]:
datasets['train'].node_label_index

tensor([     0,      1,      2,  ..., 690610, 690611, 690612], device='cuda:0')

In [81]:
datasets['val'].node_label_index

tensor([     0,      1,      2,  ..., 690610, 690611, 690612], device='cuda:0')

In [ ]:
# datasets['train'].to(args['device'])
# datasets['val'].to(args['device'])
# datasets['test'].to(args['device'])

In [8]:
losses = []

def train(model, optimizer, args):
    val_max = 0
    best_model = model

    for epoch in range(1, args['epochs'] + 1):
        datasets['train'].to(args["device"])
        model.train()
        optimizer.zero_grad()
        pred = model(datasets['train'])
        loss = model.loss(pred, datasets['train'].edge_label.type(pred.dtype))

        loss.backward()
        optimizer.step()

        log = 'Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Loss: {:.5f}, Val Loss: {:.5f}'
        score_train, train_loss = test(model, 'train', args)
        score_val, val_loss = test(model, 'val', args)
#         score_test, test_loss = test(model, 'test', args)

        losses.append((train_loss, val_loss))

        print(log.format(epoch, score_train, score_val, train_loss, val_loss))
        if val_max < score_val:
            val_max = score_val
            best_model = copy.deepcopy(model)

    return best_model

def test(model, mode, args):
    model.eval()
    score = 0
    loss_score = 0

    data = datasets[mode]
    data.to(args["device"])

    pred = model(data)
    loss = model.loss(pred, data.edge_label.type(pred.dtype))
    score += roc_auc_score(data.edge_label.flatten().cpu().numpy(), pred.flatten().data.cpu().numpy())
    loss_score += loss.item()

    return score, loss_score

In [ ]:
torch.cuda.empty_cache()

In [9]:
model = LightGCN(datasets['train'], args['num_layers'], emb_size=args['emb_size']).to(args['device'])
optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])

best_model = train(model, optimizer, args)
log = "Train: {:.4f}, Val: {:.4f}, Test: {:.4f}, Train Loss: {:.5f}, Val Loss: {:.5f}, Test Loss: {:.5f}"
best_train_roc, train_loss = test(best_model, 'train', args)
best_val_roc, val_loss = test(best_model, 'val', args)
best_test_roc, test_loss = test(best_model, 'test', args)
print(log.format(best_train_roc, best_val_roc, best_test_roc, train_loss, val_loss, test_loss))

Epoch: 001, Train: 0.6830, Val: 0.6182, Loss: 0.69299, Val Loss: 0.69640
Epoch: 002, Train: 0.8120, Val: 0.7742, Loss: 0.67594, Val Loss: 0.67927
Epoch: 003, Train: 0.8745, Val: 0.8598, Loss: 0.60988, Val Loss: 0.61604
Epoch: 004, Train: 0.8808, Val: 0.8701, Loss: 0.55814, Val Loss: 0.57589
Epoch: 005, Train: 0.8784, Val: 0.8680, Loss: 0.54047, Val Loss: 0.55366
Epoch: 006, Train: 0.8749, Val: 0.8647, Loss: 0.53826, Val Loss: 0.54585
Epoch: 007, Train: 0.8868, Val: 0.8807, Loss: 0.53550, Val Loss: 0.54033
Epoch: 008, Train: 0.8943, Val: 0.8883, Loss: 0.53690, Val Loss: 0.54510
Epoch: 009, Train: 0.8967, Val: 0.8899, Loss: 0.54031, Val Loss: 0.55114
Epoch: 010, Train: 0.9000, Val: 0.8942, Loss: 0.53715, Val Loss: 0.54548
Epoch: 011, Train: 0.8973, Val: 0.8913, Loss: 0.53634, Val Loss: 0.54254
Epoch: 012, Train: 0.8924, Val: 0.8849, Loss: 0.53537, Val Loss: 0.54202
Epoch: 013, Train: 0.8902, Val: 0.8819, Loss: 0.53301, Val Loss: 0.54155
Epoch: 014, Train: 0.8897, Val: 0.8812, Loss: 0.533

Epoch: 114, Train: 0.8952, Val: 0.8873, Loss: 0.53282, Val Loss: 0.54125
Epoch: 115, Train: 0.8951, Val: 0.8873, Loss: 0.53283, Val Loss: 0.54125
Epoch: 116, Train: 0.8951, Val: 0.8873, Loss: 0.53281, Val Loss: 0.54124
Epoch: 117, Train: 0.8951, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54122
Epoch: 118, Train: 0.8951, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54123
Epoch: 119, Train: 0.8951, Val: 0.8873, Loss: 0.53282, Val Loss: 0.54124
Epoch: 120, Train: 0.8951, Val: 0.8873, Loss: 0.53282, Val Loss: 0.54125
Epoch: 121, Train: 0.8951, Val: 0.8873, Loss: 0.53281, Val Loss: 0.54125
Epoch: 122, Train: 0.8951, Val: 0.8873, Loss: 0.53281, Val Loss: 0.54124
Epoch: 123, Train: 0.8951, Val: 0.8873, Loss: 0.53281, Val Loss: 0.54124
Epoch: 124, Train: 0.8951, Val: 0.8873, Loss: 0.53280, Val Loss: 0.54123
Epoch: 125, Train: 0.8951, Val: 0.8873, Loss: 0.53280, Val Loss: 0.54123
Epoch: 126, Train: 0.8951, Val: 0.8873, Loss: 0.53281, Val Loss: 0.54124
Epoch: 127, Train: 0.8951, Val: 0.8873, Loss: 0.532

Epoch: 227, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 228, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 229, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 230, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 231, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 232, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 233, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 234, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 235, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 236, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 237, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 238, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 239, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 240, Train: 0.8952, Val: 0.8873, Loss: 0.532

Epoch: 340, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 341, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 342, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 343, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 344, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 345, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 346, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 347, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 348, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 349, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 350, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 351, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 352, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 353, Train: 0.8952, Val: 0.8873, Loss: 0.532

Epoch: 453, Train: 0.8955, Val: 0.8877, Loss: 0.53240, Val Loss: 0.54124
Epoch: 454, Train: 0.8954, Val: 0.8877, Loss: 0.53248, Val Loss: 0.54124
Epoch: 455, Train: 0.8944, Val: 0.8863, Loss: 0.53410, Val Loss: 0.54193
Epoch: 456, Train: 0.8954, Val: 0.8877, Loss: 0.53218, Val Loss: 0.54108
Epoch: 457, Train: 0.8952, Val: 0.8874, Loss: 0.53257, Val Loss: 0.54118
Epoch: 458, Train: 0.8947, Val: 0.8866, Loss: 0.53381, Val Loss: 0.54175
Epoch: 459, Train: 0.8955, Val: 0.8878, Loss: 0.53233, Val Loss: 0.54124
Epoch: 460, Train: 0.8951, Val: 0.8873, Loss: 0.53280, Val Loss: 0.54125
Epoch: 461, Train: 0.8947, Val: 0.8867, Loss: 0.53340, Val Loss: 0.54151
Epoch: 462, Train: 0.8955, Val: 0.8877, Loss: 0.53223, Val Loss: 0.54112
Epoch: 463, Train: 0.8951, Val: 0.8872, Loss: 0.53303, Val Loss: 0.54136
Epoch: 464, Train: 0.8950, Val: 0.8870, Loss: 0.53325, Val Loss: 0.54146
Epoch: 465, Train: 0.8955, Val: 0.8877, Loss: 0.53229, Val Loss: 0.54114
Epoch: 466, Train: 0.8949, Val: 0.8870, Loss: 0.533

Epoch: 566, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 567, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 568, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 569, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 570, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 571, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 572, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 573, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 574, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 575, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 576, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 577, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 578, Train: 0.8952, Val: 0.8873, Loss: 0.53279, Val Loss: 0.54124
Epoch: 579, Train: 0.8952, Val: 0.8873, Loss: 0.532

KeyError: 'test'

In [39]:
#model_save
filename = 'best_lgcn1.pkl'
torch.save(model.state_dict(), 'model11.pth')
#datasets save
# with open('datasets_deepsnap_dict.pkl', 'wb') as f:
#     # pickle the dictionary and write it to the file
#     pickle.dump(datasets, f)

# model = torch.load('../lgcn.pkl')
# with open('../datasets_deepsnap_dict.pkl', 'rb') as f:
#     # load the pickled dictionary from the file
#     datasets = pickle.load(f)

RuntimeError: File model11.pth cannot be opened.

In [55]:
dic=model.state_dict()
torch.save(dic, 'pm.pt')

In [50]:
dic

OrderedDict([('embeddings.weight',
              tensor([[-1.5305e-02,  1.0913e-02,  6.7270e-02,  ...,  1.6636e-02,
                       -1.0406e-01, -1.8328e-04],
                      [-4.7601e-03,  3.7323e-03,  2.2234e-02,  ...,  6.0261e-03,
                       -3.4850e-02, -1.6471e-03],
                      [ 1.0785e-04, -6.6176e-04,  1.2223e-03,  ...,  3.9171e-04,
                       -8.1193e-04, -2.1851e-03],
                      ...,
                      [ 5.1203e-15,  2.0817e-14, -3.2795e-15,  ..., -5.5993e-15,
                        5.1512e-15, -5.3834e-15],
                      [ 1.4106e-06, -7.8756e-05,  4.3057e-04,  ...,  1.6524e-04,
                       -5.7623e-04,  4.2500e-05],
                      [ 1.4185e-06, -7.8778e-05,  4.3050e-04,  ...,  1.6529e-04,
                       -5.7598e-04,  4.2497e-05]], device='cuda:0'))])

In [ ]:
def plot_learning_curve(losses, title):
    train_loss, val_loss = zip(*losses)
    steps = list(range(1, len(train_loss) + 1))
    
    min_val_loss = np.round(np.min(val_loss), 3)
    # train_list = [math.log10(x) for x in train_loss]
    # val_list = [math.log10(x) for x in val_loss]
    
    plt.figure(figsize=(16, 6))
    plt.plot(steps, train_loss, '-r', label='Training Loss')
    plt.plot(steps, val_loss, '-b', label='Validation Loss')
    plt.hlines(min_val_loss, 1, 300, colors='k', linestyles='dotted', label='Min Validation Loss: {}'.format(min_val_loss))

    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    # plt.ylim((0.58, 0.71))
    plt.title(title)
    plt.legend(loc='upper right')
    plt.title(title)

    return plt

In [ ]:
plot_learning_curve(losses, 'test')

In [ ]:
data = datasets['test']
pred = model(data)

In [57]:
newModel = LightGCN(datasets['train'], args['num_layers'], emb_size=args['emb_size']).to(args['device'])

newModel.load_state_dict(torch.load('pm.pt'))

<All keys matched successfully>

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.03 GiB (GPU 0; 7.80 GiB total capacity; 4.26 GiB already allocated; 1.27 GiB free; 6.00 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
ds = datasets['test']
ds

In [ ]:
ds.edge_label

In [ ]:
type(ds.edge_label_index)

In [ ]:
dict1 = {'preds':pred.to('cpu').detach().numpy(), 'edge_label': ds.edge_label.to('cpu').detach().numpy(), 'src_node': ds.edge_label_index[0].to('cpu').detach().numpy(), 'dest_node': ds.edge_label_index[1].to('cpu').detach().numpy()}
df = pd.DataFrame(dict1)
df.head()

In [ ]:
df_0 = df[df['edge_label'] == 0]
df_0

In [ ]:
df_1 = df[df['edge_label'] == 1]
df_1

In [ ]:
df_lessthan = df_1[df_1['preds'] > 0.5]
df_lessthan

In [ ]:
df_lessthan.iloc[0]['src_node']

In [ ]:
g_nx[0]

In [ ]:
df_lessthan.iloc[0]["src_node"]
counter=0
neg=0
for x in g_nx.nodes:
    if(counter!=x):
        neg=neg+x-counter
        counter=x
    counter=counter+1
print(len(g_nx.nodes))
neg
        

In [ ]:
def row_op(row):

    if g_nx.nodes[row['src_node']]['name'] == g_nx.nodes[row['dest_node']]['name'] or g_nx.nodes[row['src_node']]['uri'] == g_nx.nodes[row['dest_node']]['uri']:
        return False
    return True

df_lessthan['is_valid'] = df_lessthan.apply(row_op,axis=1)
df_filter=df_lessthan[df_lessthan['is_valid']] 

In [ ]:
def getId(row):
    return pd.Series([g_nx.nodes[row['src_node']]['name'],g_nx.nodes[row['src_node']]['uri'],g_nx.nodes[row['src_node']]['type'],g_nx.nodes[row['dest_node']]['name'],g_nx.nodes[row['dest_node']]['uri'],g_nx.nodes[row['dest_node']]['type']])
    
df_filter[["src_name","src_uri","src_type","dest_name","dest_uri","dest_type"]]=df_filter[df_lessthan['is_valid']].apply(getId,axis=1)

In [ ]:
df_filter

In [ ]:
df_final = df_filter[(df_filter['src_name']=="Party mix") | ( df_filter['dest_name']=="Party mix") ]
df_final.sort_values('preds',ascending=False)
uri_list=list(df_final.apply(lambda x: x['src_uri'] if x['src_uri']!='' else x['dest_uri'], axis=1 ))

In [ ]:
# !pip install spotipy

In [ ]:
uri_list=[]
uri_list = list(df_final["dest_uri"])

In [ ]:
df_final

In [ ]:
uri_list

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

def gettrackname(uri_list):
    # Replace the values below with your own Spotify API credentials
    client_id = 'd5566a60926740f3a8070889731a2d21'
    client_secret = 'eb5fc0638a1241c3a611186ff8d167e3'

    # Initialize the Spotify API client
    client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

    info = []
    for uri in uri_list:
        # Use the track method to get information about the track
        track_info = sp.track(uri)

        # Get the track name from the track information
        track_name = track_info['name']
        track_info = sp.track(uri)

        # Get the artist name from the track information
        artist_name = track_info['artists'][0]['name']
        info.append((track_name, artist_name))
        
    return info

print(gettrackname(uri_list))


In [15]:
node_attrs = nx.get_node_attributes(g_nx, 'color')

NameError: name 'g_nx' is not defined